# <p style="padding:10px;background-color:#85BB65;margin:0;color:white;font-family:newtimeroman;font-size:150%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">Imputation methods</p>


Imputation is a technique used to fill in missing values in a dataset. Missing values are gaps or unknown entries in the data that can occur for various reasons such as ranging from human errors during data entry, incorrect sensor readings, to software bugs in the data processing pipeline.

Imputation methods aim to estimate the missing values based on the available information in the dataset.

In [1]:
# modules we'll use
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer


In [2]:
# read in all our data
data = pd.read_csv(r'C:\Users\tawfi\projects\clean data tutorial\Imputation\NFL.csv', low_memory=False)

# set seed for reproducibility
np.random.seed(0) 

The first thing to do when you get a new dataset is take a look at some of it. This lets you see that it all read in correctly and gives an idea of what's going on with the data. In this case, let's see if there are any missing values, which will be reprsented with NaN or None.

In [3]:
data.head()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2009-09-10,2009091000,1,1,NaN,15:00,15,3600.0,0.0,TEN,...,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
1,2009-09-10,2009091000,1,1,1.0,14:53,15,3593.0,7.0,PIT,...,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,2009-09-10,2009091000,1,1,2.0,14:16,15,3556.0,37.0,PIT,...,NaN,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,NaN,NaN,2009
3,2009-09-10,2009091000,1,1,3.0,13:35,14,3515.0,41.0,PIT,...,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2009-09-10,2009091000,1,1,4.0,13:27,14,3507.0,8.0,PIT,...,NaN,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,NaN,NaN,2009


In [4]:
# get the number of missing data points per column
print(f"all missing values in data: {data.isna().sum().sum()}")
print()
print(data.isnull().sum().head(10))


all missing values in data: 9190078

Date                0
GameID              0
Drive               0
qtr                 0
down            54218
time              188
TimeUnder           0
TimeSecs          188
PlayTimeDiff      374
SideofField       450
dtype: int64


In [5]:
# columns which have nulls and the percentage of nulls in each column

data_na = (data.isnull().sum() / len(data)) 
data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'mean of nulls' :data_na , "number_of_nulls" : data[data_na.index].isna().sum()})
missing_data.head(10)

,mean of nulls,number_of_nulls
DefTwoPoint,0.999961,362433
BlockingPlayer,0.999708,362341
TwoPointConv,0.998543,361919
ChalReplayResult,0.991803,359476
RecFumbPlayer,0.989146,358513
RecFumbTeam,0.989146,358513
Interceptor,0.988798,358387
FieldGoalDistance,0.978151,354528
FieldGoalResult,0.977884,354431
ExPointResult,0.975036,353399


In [6]:
missing_data.tail(10)

,mean of nulls,number_of_nulls
TimeSecs,0.000519,188
Touchdown_Prob,0.000408,148
ExpPts,0.000408,148
Safety_Prob,0.000408,148
Opp_Touchdown_Prob,0.000408,148
Opp_Safety_Prob,0.000408,148
Opp_Field_Goal_Prob,0.000408,148
No_Score_Prob,0.000408,148
Field_Goal_Prob,0.000408,148
desc,0.000006,2


* That seems like a lot! It might be helpful to see what percentage of the values in our dataset were missing to give us a better sense of the scale of this problem:

In [7]:
# how many total missing values do we have?
total_cells = np.product(data.shape)
total_missing = data.isna().sum().sum()

percent_missing = (total_missing/total_cells) * 100
print(f'percent of data that is missing equal: {round(percent_missing ,3)}')
print(f'total missing values equal: {total_missing}')

percent of data that is missing equal: 24.858
total missing values equal: 9190078


* almost a quarter of the cells in this dataset are empty! 

# **<span style='color:#85BB65'>why the value is missing! </span>**

<br>

you'll need to use your intution to figure out why the value is missing. One of the most important questions you can ask yourself to help figure this out is this:


<br>


**Is this value missing because it wasn't recorded or because it doesn't exist?**

<mark style="background-color:#85BB65;color:white;font-size:100%;border-radius:3px;opacity:2.0">1</mark> If a value is missing becuase it doesn't exist (like the height of the oldest child of someone who doesn't have any children) then it doesn't make sense to try and guess what it might be. These values you probably do want to keep as NaN.

<mark style="background-color:#85BB65;color:white;font-size:100%;border-radius:3px;opacity:2.0">2</mark> if a value is missing because it wasn't recorded, then you can try to guess what it might have been based on the other values in that column and row. This is called imputation.

By looking at the data information, you can see that `TimeSecs` column has information on the number of seconds left in the game when the play was made. This means that these values are probably missing because they were not recorded, rather than because they don't exist. So, it would make sense for us to try and guess what they should be rather than just leaving them as NA's.

On the other hand `PenalizedTeam`  column that also have lot of missing fields. In this case the field is missing because if there was no penalty then it doesn't make sense to say which team was penalized. For this column, it would make more sense to either leave it empty or to add a third value like "neither" and use that to replace the NA's.

<div style="border-radius:10px;border:#85BB65 solid;padding: 15px;background-color:#ffffff00;font-size:100%;text-align:left">
If you're doing very careful data analysis, this is the point at which you'd look at each column individually to figure out the best strategy for filling those missing values.
</div>

# **<span style='color:#85BB65'>How to deal with missing values? </span>**

## <mark style="background-color:#85BB65;color:white;font-size:120%;border-radius:3px;opacity:2.0">1- Drop columns with more than half of its data is missing  </mark>

<br>

Columns with more than half of their data are empty, do not give enough information for our model.
A general rule is that, if more than half of the data in a column is missing, it's better to drop it then try imputing.

In [8]:
droped_col=[]
for col in data.columns:
        
    # drop columns that are more than half nan-values.
    if data[col].isna().sum()/len(data) > 0.5 :
        droped_col.append(col)
        data.drop(columns=col , inplace = True)

In [9]:
print(data.shape)
print()
print(len(droped_col))

(362447, 73)

29


* we have 29 columns that have more than half nan-values so we dropped them.

## <mark style="background-color:#85BB65;color:white;font-size:120%;border-radius:3px;opacity:2.0">2- Drop missing values  </mark>


if you don't have a reason to figure out why your values are missing, one option you have is to just remove any rows or columns that contain missing values. I don't generally recommend this approch, becouse it couses a problem of leakage of data and may lead to underfitting.

In [10]:
data.dropna()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,TwoPoint_Prob,ExpPts,EPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,Season
1,2009-09-10,2009091000,1,1,1.0,14:53,15,3593.0,7.0,PIT,...,0.0,2.338000,0.077907,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,2009
2,2009-09-10,2009091000,1,1,2.0,14:16,15,3556.0,37.0,PIT,...,0.0,2.415907,-1.402760,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,2009
5,2009-09-10,2009091000,2,1,1.0,13:16,14,3496.0,11.0,TEN,...,0.0,-1.398360,-0.696302,0.558929,0.441071,0.578453,0.421547,0.441071,-0.019524,2009
6,2009-09-10,2009091000,2,1,2.0,12:40,13,3460.0,36.0,TEN,...,0.0,-2.094662,-0.179149,0.578453,0.421547,0.582881,0.417119,0.421547,-0.004427,2009
7,2009-09-10,2009091000,2,1,3.0,12:11,13,3431.0,29.0,TEN,...,0.0,-2.273811,-1.119477,0.582881,0.417119,0.617544,0.382456,0.417119,-0.034663,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362428,2017-01-01,2017010102,18,4,2.0,02:46,3,166.0,4.0,DET,...,0.0,0.267123,0.972470,0.019621,0.980379,0.019667,0.980333,0.019621,0.000047,2016
362429,2017-01-01,2017010102,18,4,1.0,02:24,3,144.0,22.0,DET,...,0.0,1.239593,1.079178,0.019667,0.980333,0.020248,0.979752,0.019667,0.000581,2016
362430,2017-01-01,2017010102,18,4,1.0,02:04,3,124.0,20.0,GB,...,0.0,2.318771,1.020103,0.020248,0.979752,0.022517,0.977483,0.020248,0.002269,2016
362433,2017-01-01,2017010102,18,4,2.0,01:53,2,113.0,4.0,GB,...,0.0,3.025061,-0.153656,0.019399,0.980601,0.013864,0.986136,0.019399,-0.005535,2016


* it looks like that's removed all our data! This is because every row in our dataset had at least one missing value.

## <mark style="background-color:#85BB65;color:white;font-size:120%;border-radius:3px;opacity:2.0">3- Imputing </mark>

<br>

Imputation methods aim to estimate the missing values based on the available information in the dataset. The choice of imputation method depends on the nature of the data and the specific analysis or modeling task at hand. 


## <b>I <span style='color:#85BB65'>|</span> useing pandas </b> 

<br>

replace missing values with whatever value comes directly after it in the same column. This makes a lot of sense for datasets where the observations have some sort of logical order to them.

In [11]:
df1 = data.copy()

In [12]:
def deal_with_nulls( dataframe , columns = None ):
        
    if columns is None:
        columns = dataframe.columns
        
    for col in dataframe.columns:
        
        # replace all NA's the value that comes directly after it in the same column, 
        # then replace all the remaining na's with suitable value.
        
        if dataframe[col].dtype =="float":
            dataframe[col] = dataframe[col].fillna(method='bfill', axis=0).fillna(dataframe[col].mean())

        elif dataframe[col].dtype =="int":
            dataframe[col] = dataframe[col].fillna(method='bfill', axis=0).fillna(dataframe[col].median())

        elif dataframe[col].dtype =="object":
            dataframe[col] = dataframe[col].fillna(method='bfill', axis=0).fillna(dataframe[col].mode()[0])

    
    return dataframe.head()

In [13]:
deal_with_nulls(df1)

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,TwoPoint_Prob,ExpPts,EPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,Season
0,2009-09-10,2009091000,1,1,1.0,15:00,15,3600.0,0.0,TEN,...,0.0,0.323526,2.014474,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,2009
1,2009-09-10,2009091000,1,1,1.0,14:53,15,3593.0,7.0,PIT,...,0.0,2.338000,0.077907,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,2009
2,2009-09-10,2009091000,1,1,2.0,14:16,15,3556.0,37.0,PIT,...,0.0,2.415907,-1.402760,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,2009
3,2009-09-10,2009091000,1,1,3.0,13:35,14,3515.0,41.0,PIT,...,0.0,1.013147,-1.712583,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,2009
4,2009-09-10,2009091000,1,1,4.0,13:27,14,3507.0,8.0,PIT,...,0.0,-0.699436,2.097796,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,2009


In [14]:
print(f"all missing values in data {df1.isna().sum().sum()}")
print()
print(df1.isnull().sum().head(10))

all missing values in data 0

Date            0
GameID          0
Drive           0
qtr             0
down            0
time            0
TimeUnder       0
TimeSecs        0
PlayTimeDiff    0
SideofField     0
dtype: int64


## <b>II <span style='color:#85BB65'>|</span> useing sklearn </b> 


In [15]:
df2 = data.copy()

In [16]:
def impute_nulls_value(columns , strategy):
    
    # Create an instance of SimpleImputer with the desired strategy
    imputer = SimpleImputer(strategy= strategy)

    
    # Fit and transform the selected columns
    df2[columns] = imputer.fit_transform(df2[columns])

**You can choose the most appropriate strategy based on the nature of your data and the desired imputation approach.**
* `mean` : Replaces missing values with the mean of the non-missing values in the column.
* `median` : Replaces missing values with the median of the non-missing values in the column.
* `most_frequent` : Replaces missing values with the most frequent value (mode) in the column.

In [17]:
df2[['Home_WP_pre','Away_WP_pre', 'Home_WP_post']].isna().sum()

Home_WP_pre     21943
Away_WP_pre     21943
Home_WP_post    23360
dtype: int64

In [18]:
# impute some columns
impute_nulls_value(['Home_WP_pre','Away_WP_pre', 'Home_WP_post'] , strategy="mean")

In [19]:
#check
df2[['Home_WP_pre','Away_WP_pre', 'Home_WP_post']].isna().sum()

Home_WP_pre     0
Away_WP_pre     0
Home_WP_post    0
dtype: int64

that is very beautiful . we now learn how to deal with null values.

***

<br>

<div style="text-align: center;">
   <span style="font-size: 4.5em; font-weight: bold; font-family: Arial;">THANK YOU!</span>
</div>

<br>
<br>

<div style="text-align: center;">
    <span style="font-size: 5em;">✔️</span>
</div>

<br>

<div style="text-align: center;">
   <span style="font-size: 1.4em; font-weight: bold; font-family: Arial; max-width:1200px; display: inline-block;">
       I hope you find this article helpful and have learned some new things ❤
       If you find this notebook useful, I'd greatly appreciate your upvote!
   </span>
</div>
